# Прунинг [SegFormer](https://huggingface.co/docs/transformers/model_doc/segformer)

Будем прунить SegFormer для [задачи сегментации людей](https://www.kaggle.com/datasets/laurentmih/aisegmentcom-matting-human-datasets).

Cам датасет [сегментации](https://drive.google.com/file/d/1YOEDzZvhLb2DS1Yn7p7MSs41ou3ZBXUq/view?usp=sharing)

## Установим библиотеки

In [ ]:
!pip install torch transformers tensorboard pillow torch_pruning --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets==v2.11.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00


Смонтируем гугл диск

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/distill/

Mounted at /content/drive
/content/drive/MyDrive/distill


Уберем варнинги

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import os

import typing
import torch

from copy import deepcopy
from datasets import load_metric
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard.writer import SummaryWriter
from tqdm.auto import tqdm

from utils.data import init_dataloaders
from utils.model import evaluate_model
from utils.model import init_model_with_pretrain

from torch import nn
from transformers.models.segformer.modeling_segformer import SegformerLayer, SegformerEfficientSelfAttention

import torch_pruning as tp

In [ ]:
baseline_path = '/content/drive/MyDrive/distill/hw_files_2/runs/baseline_ckpt.pth'
distilled_ckpt = '/content/drive/MyDrive/distill/hw_files_2/runs/distillation/ckpt_2.pth'

In [ ]:
# маппинг названия классов и индексов
id2label = {
    0: "background",
    1: "human",
}
label2id = {v: k for k, v in id2label.items()}

Создадим лоадеры:

In [ ]:
train_dataloader, valid_dataloader = init_dataloaders(
    root_dir=".",
    batch_size=16,
    num_workers=8,
)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Создадим baseline модель:

In [ ]:
baseline_model = init_model_with_pretrain(label2id=label2id, id2label=id2label, pretrain_path=baseline_path).cuda()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/70.0k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/14.4M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/14.3M [00:00<?, ?B/s]

И сразу отвалидируем:

In [ ]:
evaluate_model(baseline_model, valid_dataloader, id2label)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Mean_iou: 0.9859687040611708
Mean accuracy: 0.9929542922652376


{'mean_iou': 0.9859687040611708,
 'mean_accuracy': 0.9929542922652376,
 'overall_accuracy': 0.9929354332930205,
 'per_category_iou': array([0.98610264, 0.98583477]),
 'per_category_accuracy': array([0.9912933 , 0.99461528])}

Создадим модель после дистилляции

In [ ]:
distilled_model = init_model_with_pretrain(label2id=label2id, id2label=id2label, pretrain_path=distilled_ckpt).cuda()

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Проверим точность:

In [ ]:
evaluate_model(distilled_model, valid_dataloader, id2label)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Mean_iou: 0.9779563962888942
Mean accuracy: 0.9888828785258961


{'mean_iou': 0.9779563962888942,
 'mean_accuracy': 0.9888828785258961,
 'overall_accuracy': 0.9888562335083816,
 'per_category_iou': array([0.97815001, 0.97776278]),
 'per_category_accuracy': array([0.98653614, 0.99122962])}

Оценим вычислительную сложность и количество параметров моделей:

In [ ]:
input_example = torch.rand(1,3,512,512, device="cuda")

In [ ]:
ops, params = tp.utils.count_ops_and_params(baseline_model, input_example)
print(f"Baseline model complexity: {ops/1e6} MMAC, {params/1e6} M params")

Baseline model complexity: 6761.228288 MMAC, 3.714658 M params


In [ ]:
ops, params = tp.utils.count_ops_and_params(distilled_model, input_example)
print(f"Distilled model complexity: {ops/1e6} MMAC, {params/1e6} M params")

Distilled model complexity: 5841.819136 MMAC, 2.29821 M params


Проверим, что модель после дистилляции имеет по одному SegformerLayer в block-е:

In [ ]:
distilled_model

## Magnitude pruning

Выполним one-shot прунинг модели по L2 норме весов. Не будем прунить последний слой:

In [ ]:
def prune_model_l2(model, pruning_ratio = 0.5, pruning_type = 'l2'):

  example_inputs = torch.rand(1,3,512,512, device="cuda")
  last_layers_id = len(list(model.modules()))

  ignored_layers = []
  for name, module in model.named_modules():

    # не будем трогать голову
    if 'decode_head' in name:
      ignored_layers.append(module)

  if pruning_type == 'random':
    imp = tp.importance.RandomImportance()
  elif pruning_type == 'taylor':
      imp = tp.importance.TaylorImportance()
  elif pruning_type == 'l1':
      imp = tp.importance.MagnitudeImportance(p=1)
  elif pruning_type == 'l2':
      imp = tp.importance.MagnitudeImportance(p=2)

  else: raise NotImplementedError

  pruner = tp.pruner.MetaPruner(
                  model = model,
                  example_inputs = example_inputs,
                  global_pruning= False, # If False, a uniform pruning ratio will be assigned to different layers.
                  importance=imp,
                  pruning_ratio = pruning_ratio,
                  ignored_layers=ignored_layers,
  )

  pruner.step()
  return model

pruned_distilled_model = prune_model_l2(deepcopy(distilled_model))

Заметим снижение числа каналов в линейном слое

In [ ]:
pruned_distilled_model

In [ ]:
# Проверим, запускается ли наша запруненная сеть
pruned_distilled_model(input_example);

RuntimeError: shape '[1, 16384, 1, 32]' is invalid for input of size 262144

In [ ]:
# Посмотрим в чем дело
for stage in pruned_distilled_model.segformer.encoder.block:
  for layer in stage:
      hidden_size = layer.layer_norm_1.normalized_shape[0]
      head_att = layer.attention.self

      print('Размер канала после прунинга ', hidden_size)
      print('Размер канала головы ', head_att.attention_head_size)
      print('Сумма каналов голов',head_att.all_head_size)
      print('Всего голов', head_att.num_attention_heads)
      print()

Размер канала после прунинга  16
Размер канала головы  32
Сумма каналов голов 32
Всего голов 1

Размер канала после прунинга  32
Размер канала головы  32
Сумма каналов голов 64
Всего голов 2

Размер канала после прунинга  80
Размер канала головы  32
Сумма каналов голов 160
Всего голов 5

Размер канала после прунинга  128
Размер канала головы  32
Сумма каналов голов 256
Всего голов 8



### Чиним модель

После прунинга у модели снизилось число каналов в голове но в слое внимания, поправим это:

In [ ]:
# Проанализируйте лог ошибки, и поймите почему модель перестала запускаться после прунинга
# Подсказка, это связано со слоем внимания и размером голов
def fix_attention_layer(pruned_model):
  for stage in pruned_model.segformer.encoder.block:
    for layer in stage:

      # Получаем текущий hidden_size
      hidden_size = layer.layer_norm_1.normalized_shape[0]
      head_att = layer.attention.self

      # Обновляем параметры в Segformer
      head_att.attention_head_size = hidden_size // head_att.num_attention_heads
      head_att.all_head_size = head_att.num_attention_heads * head_att.attention_head_size

  return pruned_model

In [ ]:
pruned_distilled_model = fix_attention_layer(pruned_distilled_model)

In [ ]:
for stage in pruned_distilled_model.segformer.encoder.block:
  for layer in stage:
      hidden_size = layer.layer_norm_1.normalized_shape[0]
      head_att = layer.attention.self

      print('Размер канала после прунинга ', hidden_size)
      print('Размер канала головы ', head_att.attention_head_size)
      print('Сумма каналов голов',head_att.all_head_size)
      print('Всего голов', head_att.num_attention_heads)
      print()

Размер канала после прунинга  16
Размер канала головы  16
Сумма каналов голов 16
Всего голов 1

Размер канала после прунинга  32
Размер канала головы  16
Сумма каналов голов 32
Всего голов 2

Размер канала после прунинга  80
Размер канала головы  16
Сумма каналов голов 80
Всего голов 5

Размер канала после прунинга  128
Размер канала головы  16
Сумма каналов голов 128
Всего голов 8



In [ ]:
# Убедимся, что модель запускается после фикса
pruned_distilled_model(input_example)

In [ ]:
# Оценим вычислительную сложность получившейся модели
ops, params = tp.utils.count_ops_and_params(distilled_model, input_example)
print(f"Distilled model complexity (Before magnitude pruning): {ops/1e6} MMAC, {params/1e6} M params")

ops, params = tp.utils.count_ops_and_params(pruned_distilled_model, input_example)
print(f"Distilled model complexity (After magnitude pruning): {ops/1e6} MMAC, {params/1e6} M params")

Distilled model complexity (Before magnitude pruning): 5841.819136 MMAC, 2.29821 M params
Distilled model complexity (After magnitude pruning): 4794.207488 MMAC, 0.814258 M params


Разумеется просела точность

In [ ]:
evaluate_model(pruned_distilled_model, valid_dataloader, id2label)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Mean_iou: 0.25392953732474766
Mean accuracy: 0.5006966812125788


{'mean_iou': 0.25392953732474766,
 'mean_accuracy': 0.5006966812125788,
 'overall_accuracy': 0.5063590649737427,
 'per_category_iou': array([0.50587387, 0.00198521]),
 'per_category_accuracy': array([0.99940695, 0.00198641])}

In [ ]:
# Попробуем уменьшать модель еще сильнее, запрунив головы в attention.
# Функционал torch pruning это не поддерживает, однако это доступно в transformers
# Для выбора наименее полезных голов можно воспользоваться L2 нормой весов.
# Мы же тут выкинем все, кроме нулевой.

pruned_distilled_model.segformer.encoder.block[1][0].attention.prune_heads([1])
pruned_distilled_model.segformer.encoder.block[2][0].attention.prune_heads([1,2,3,4])
pruned_distilled_model.segformer.encoder.block[3][0].attention.prune_heads([1,2,3,4,5,6,7])

In [ ]:
for stage in pruned_distilled_model.segformer.encoder.block:
  for layer in stage:
      hidden_size = layer.layer_norm_1.normalized_shape[0]
      head_att = layer.attention.self

      print('Размер канала после прунинга ', hidden_size)
      print('Размер канала головы ', head_att.attention_head_size)
      print('Сумма каналов голов',head_att.all_head_size)
      print('Всего голов', head_att.num_attention_heads)
      print()

Размер канала после прунинга  16
Размер канала головы  16
Сумма каналов голов 16
Всего голов 1

Размер канала после прунинга  32
Размер канала головы  16
Сумма каналов голов 16
Всего голов 1

Размер канала после прунинга  80
Размер канала головы  16
Сумма каналов голов 16
Всего голов 1

Размер канала после прунинга  128
Размер канала головы  16
Сумма каналов голов 16
Всего голов 1



In [ ]:
# Снова оценим вычислительную сложность
ops, params = tp.utils.count_ops_and_params(pruned_distilled_model, input_example)
print(f"Distilled model complexity (After magnitude pruning): {ops/1e6} MMAC, {params/1e6} M params")

Distilled model complexity (After magnitude pruning): 4761.9632 MMAC, 0.73381 M params


##  Дообучение запруненной модели

#### Прошлый код

In [ ]:
from dataclasses import dataclass
import typing as typing

@dataclass
class TrainParams:
    n_epochs: int
    lr: float
    batch_size: int
    n_workers: int
    device: torch.device
    temperature: int

    loss_weight: float
    last_layer_loss_weight: float
    intermediate_layers_weights: typing.Tuple[float, float, float, float]
    intermediate_attn_layers_weights: typing.Tuple[float, float, float, float]
    intermediate_feat_layers_weights: typing.Tuple[float, float, float, float]

In [ ]:
mse_loss = nn.MSELoss()
kl_loss = nn.KLDivLoss()

def calc_last_layer_loss(student_logits, teacher_logits, weight):
    return mse_loss(student_logits, teacher_logits) * weight

def calc_intermediate_layers_attn_loss(student_logits, teacher_logits, weights, student_teacher_attention_mapping):
    return None

def calc_intermediate_layers_feat_loss(student_feats, teacher_feats, weights):
  return None


def calc_intermediate_layers_loss(student_attentions, teacher_attentions, weights, student_teacher_attention_mapping):
    intermediate_kl_loss = 0
    for i, (stud_attn_idx, teach_attn_idx) in enumerate(student_teacher_attention_mapping.items()):
        intermediate_kl_loss += weights[i] * kl_loss(
            input=torch.log(student_attentions[stud_attn_idx]),
            target=teacher_attentions[teach_attn_idx],
        )
    return intermediate_kl_loss

In [ ]:
def train(
    teacher_model,
    student_model,
    train_params: TrainParams,
    student_teacher_attention_mapping,
    tb_writer,
    save_dir
):
    metric = load_metric('mean_iou')
    teacher_model.to(train_params.device)
    student_model.to(train_params.device)

    teacher_model.eval()

    train_dataloader, valid_dataloader = init_dataloaders(
        root_dir=".",
        batch_size=train_params.batch_size,
        num_workers=train_params.n_workers,
    )

    optimizer = torch.optim.AdamW(student_model.parameters(), lr=train_params.lr)
    step = 0
    for epoch in range(train_params.n_epochs):
        pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        for idx, batch in pbar:
            student_model.train()

            pixel_values = batch['pixel_values'].to(train_params.device)
            labels = batch['labels'].to(train_params.device)

            optimizer.zero_grad()

            # forward + backward + optimize
            student_outputs = student_model(
                pixel_values=pixel_values,
                labels=labels,
                output_attentions=True,
                output_hidden_states=True,
            )
            loss, student_logits = student_outputs.loss, student_outputs.logits

            with torch.no_grad():
                teacher_output = teacher_model(
                    pixel_values=pixel_values,
                    labels=labels,
                    output_attentions=True,
                    output_hidden_states=True,
                )


            last_layer_loss = calc_last_layer_loss(
                student_logits,
                teacher_output.logits,
                train_params.last_layer_loss_weight,
            )

            student_attentions, teacher_attentions = student_outputs.attentions, teacher_output.attentions
            student_hidden_states, teacher_hidden_states = student_outputs.hidden_states, teacher_output.hidden_states

            intermediate_layer_att_loss = calc_intermediate_layers_attn_loss(
                student_attentions,
                teacher_attentions,
                train_params.intermediate_attn_layers_weights,
                student_teacher_attention_mapping,
            )

            intermediate_layer_feat_loss = calc_intermediate_layers_feat_loss(
                student_hidden_states,
                teacher_hidden_states,
                train_params.intermediate_feat_layers_weights,
            )

            total_loss = loss* train_params.loss_weight + last_layer_loss
            if intermediate_layer_att_loss is not None:
                total_loss += intermediate_layer_att_loss

            if intermediate_layer_feat_loss is not None:
                total_loss += intermediate_layer_feat_loss

            step += 1

            total_loss.backward()
            optimizer.step()
            pbar.set_description(f'total loss: {total_loss.item():.3f}')

            for loss_value, loss_name in (
                (loss, 'loss'),
                (total_loss, 'total_loss'),
                (last_layer_loss, 'last_layer_loss'),
                (intermediate_layer_att_loss, 'intermediate_layer_att_loss'),
                (intermediate_layer_feat_loss, 'intermediate_layer_feat_loss'),
            ):
                if loss_value is None: # для выключенной дистилляции атеншенов
                    continue
                tb_writer.add_scalar(
                    tag=loss_name,
                    scalar_value=loss_value.item(),
                    global_step=step,
                )

        #после модификаций модели обязательно сохраняйте ее целиком, чтобы подгрузить ее в случае чего
        torch.save(
            {
                'model': student_model,
                'state_dict': student_model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
            },
            f'{save_dir}/ckpt_{epoch}.pth',
        )

        eval_metrics = evaluate_model(student_model, valid_dataloader, id2label)

        for metric_key, metric_value in eval_metrics.items():
            if not isinstance(metric_value, float):
                continue
            tb_writer.add_scalar(
                tag=f'eval_{metric_key}',
                scalar_value=metric_value,
                global_step=epoch,
            )


#### Обучение

In [ ]:
train_params = TrainParams(
    n_epochs=1,
    lr=1e-4,
    batch_size=16,
    n_workers=8,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    temperature=10,
    loss_weight=0.5,
    last_layer_loss_weight=0.5,
    intermediate_attn_layers_weights = (0.5, 0.5, 0.5, 0.5),
    intermediate_feat_layers_weights = (0.5, 0.5, 0.5, 0.5),
)

In [ ]:
save_dir = 'runs/magnitude_equal_pruning'
tb_writer = SummaryWriter(save_dir)

In [ ]:
with torch.no_grad():
    teacher_attentions = baseline_model(pixel_values=torch.ones(1, 3, 512, 512).to(train_params.device), output_attentions=True).attentions
    student_attentions = pruned_distilled_model(pixel_values=torch.ones(1, 3, 512, 512).to(train_params.device), output_attentions=True).attentions

In [ ]:
print(f"{len(teacher_attentions)} слоев учителя")
print(f"Против {len(student_attentions)} слоев учителя")


8 слоев учителя
Против 4 слоев учителя


In [ ]:
student_teacher_attention_mapping = {0: 1, 1: 3, 2: 5, 3: 7}

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

Mean_iou: 0.9202542951793572
Mean accuracy: 0.9585926503382349

In [ ]:
train(
    teacher_model=baseline_model,
    student_model=deepcopy(pruned_distilled_model),
    train_params=train_params,
    student_teacher_attention_mapping=student_teacher_attention_mapping,
    tb_writer=tb_writer,
    save_dir=save_dir,
)

## Taylor pruning

Теперь выполним прунинг по Taylor критерию важности, и сравним точности полученных моделей после тюнинга. Уровень прунинга и структуру оствим такой же, как для L2 для чистоты эксперимента.

In [ ]:
distilled_model = init_model_with_pretrain(label2id=label2id, id2label=id2label, pretrain_path=distilled_ckpt).cuda()

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def prune_model_taylor(model, pruning_ratio=0.5):
  example_inputs = torch.rand(1,3,512,512, device="cuda")
  last_layers_id = len(list(model.modules()))

  ignored_layers = []
  for name, module in model.named_modules():
    # не будем трогать голову
    if 'decode_head' in name:
      ignored_layers.append(module)

  imp = tp.importance.TaylorImportance()

  pruner = tp.pruner.MetaPruner(
                  model = model,
                  example_inputs = example_inputs,
                  global_pruning= False,
                  importance=imp,
                  pruning_ratio = pruning_ratio,
                  ignored_layers=ignored_layers,
  )
  return pruner

pruner = prune_model_taylor(distilled_model)

In [ ]:
print("Критерий:", type(pruner.importance).__name__)
print("Уровень прунинга:", pruner.pruning_ratio)
print("Игнорируемые слои:", [m.__class__.__name__ for m in pruner.ignored_layers])
print("Глобальный прунинг:", pruner.global_pruning)

Критерий: TaylorImportance
Уровень прунинга: 0.5
Игнорируемые слои: ['SegformerDecodeHead', 'ModuleList', 'SegformerMLP', 'Linear', 'SegformerMLP', 'Linear', 'SegformerMLP', 'Linear', 'SegformerMLP', 'Linear', 'Conv2d', 'BatchNorm2d', 'ReLU', 'Dropout', 'Conv2d', 'ModuleList', 'SegformerMLP', 'Linear', 'SegformerMLP', 'Linear', 'SegformerMLP', 'Linear', 'SegformerMLP', 'Linear', 'SegformerMLP', 'Linear', 'Linear', 'SegformerMLP', 'Linear', 'Linear', 'SegformerMLP', 'Linear', 'Linear', 'SegformerMLP', 'Linear', 'Linear', 'Conv2d', 'BatchNorm2d', 'ReLU', 'Dropout', 'Conv2d']
Глобальный прунинг: False


Для прунинга по Тейлору необходимо накопить градиенты на весах, они используются для оценки важности каналов

In [ ]:
def calibrate_model(distilled_model, valid_dataloader):
  distilled_model.train()  # Режим обучения для градиентов
  distilled_model.zero_grad()

  num_batches_for_grad = 3  # Число батчей для усреднения
  valid_iter = iter(valid_dataloader)

  for _ in range(num_batches_for_grad):
      try:
          batch = next(valid_iter)
      except StopIteration:
          valid_iter = iter(valid_dataloader)
          batch = next(valid_iter)

      pixel_values = batch['pixel_values'].to(train_params.device)
      labels = batch['labels'].to(train_params.device)

      # Forward pass
      student_outputs = distilled_model(
          pixel_values=pixel_values,
          labels=labels,
          output_attentions=True,
          output_hidden_states=True,
      )
      loss = student_outputs.loss
      loss = loss / num_batches_for_grad  # Нормализация для усреднения
      loss.backward()  # Аккумуляция градиентов

  return distilled_model

Применение прунинга и его создание разнесены по функциям:

In [2]:
def apply_taylor_pruning(pruner):
  pruner.step()
  return None

In [ ]:
calibrate_model(distilled_model, valid_dataloader)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
apply_taylor_pruning(pruner)

In [ ]:
# Проверим, запускается ли наша запруненная сеть
distilled_model(input_example);

In [ ]:
distilled_model = fix_attention_layer(distilled_model)

Тот же фикс, как для прунинга по L2

In [ ]:
distilled_model(input_example)

SemanticSegmenterOutput(loss=None, logits=tensor([[[[ 3.1855,  3.9675,  3.1165,  ...,  5.4188,  5.6611,  5.2579],
          [ 3.2630,  4.6218,  5.1183,  ...,  6.5539,  7.5736,  5.2526],
          [ 2.4518,  4.6016,  6.2250,  ...,  6.1666,  5.5939,  5.3026],
          ...,
          [-1.2342, -1.9257, -1.9217,  ...,  2.9774,  3.2960,  2.8177],
          [-1.1975, -0.0352, -1.5325,  ...,  3.6424,  3.0609,  2.7927],
          [-0.2898, -0.6798, -1.2796,  ...,  2.8469,  4.0563,  4.0721]],

         [[-3.8411, -5.1103, -3.7023,  ..., -4.6261, -5.3411, -4.7295],
          [-3.9321, -5.3087, -5.7201,  ..., -6.3317, -6.6841, -4.8200],
          [-3.0980, -5.1427, -6.0533,  ..., -5.7867, -4.6713, -4.6689],
          ...,
          [ 0.6650,  1.1572,  1.6151,  ..., -3.1517, -3.1056, -2.7185],
          [ 0.4760, -0.6740,  0.9841,  ..., -3.3024, -2.8670, -2.5201],
          [-0.4292, -0.1965,  0.5277,  ..., -3.0930, -3.9383, -3.8231]]]],
       device='cuda:0', grad_fn=<ConvolutionBackward0>), hi

In [ ]:
# Изначальная модель
ops, params = tp.utils.count_ops_and_params(baseline_model, input_example)
print(f"Original model complexity (After taylor pruning): {ops/1e6} MMAC, {params/1e6} M params")

Original model complexity (After taylor pruning): 6761.228288 MMAC, 3.714658 M params


In [ ]:
# Оценим сложность полученной модели
ops, params = tp.utils.count_ops_and_params(distilled_model, input_example)
print(f"Distilled model complexity (After taylor pruning): {ops/1e6} MMAC, {params/1e6} M params")

Distilled model complexity (After taylor pruning): 4794.207488 MMAC, 0.814258 M params


In [ ]:
# После L2
ops, params = tp.utils.count_ops_and_params(pruned_distilled_model, input_example)
print(f"Distilled model complexity (After magnitude pruning): {ops/1e6} MMAC, {params/1e6} M params")

Distilled model complexity (After magnitude pruning): 4761.9632 MMAC, 0.73381 M params


Выполним дообучение, и сравним точности

In [ ]:
train_params = TrainParams(
    n_epochs=2,
    lr=1e-4,
    batch_size=16,
    n_workers=8,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    temperature=10,
    loss_weight=0.5,
    last_layer_loss_weight=0.5,
    intermediate_attn_layers_weights = (0.5, 0.5, 0.5, 0.5),
    intermediate_feat_layers_weights = (0.5, 0.5, 0.5, 0.5),
)

In [ ]:
save_dir = 'runs/taylor_equal_pruning'
tb_writer = SummaryWriter(save_dir)

In [ ]:
train(
    teacher_model=baseline_model,
    student_model=deepcopy(distilled_model),
    train_params=train_params,
    student_teacher_attention_mapping=student_teacher_attention_mapping,
    tb_writer=tb_writer,
    save_dir=save_dir,
)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


  0%|          | 0/1129 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Mean_iou: 0.9432862131851423
Mean accuracy: 0.9708174277482802


  0%|          | 0/1129 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Mean_iou: 0.9558181386991196
Mean accuracy: 0.9774450762901064


После одной эпохи точность у **Тейлора**
```
Mean_iou: 0.9424630083220478
Mean accuracy: 0.9704968695467675

```

У **L2**


```
Mean_iou: 0.9202542951793572
Mean accuracy: 0.9585926503382349
```




# Итог

После всех ухишрений
точность дистилированной и запруненой сетки после дообучения на 4 эпохах:

```
Mean_iou: 0.9558181386991196
Mean accuracy: 0.9774450762901064
```




Оригинальная модель из статьи

```
Mean_iou: 0.9859687040611708
Mean accuracy: 0.9929542922652376
```




Снизив число параметров в **4.5** раз

3.714658M / 0.814258 M

4794.207488 MMAC, 0.814258 M params
6761.228288 MMAC, 3.714658 M params